In [15]:
import numpy as np
import pandas as pd
import theano as T
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score 

In [19]:
xls_file = pd.ExcelFile('horly_corn_log_reg.xlsx')

In [20]:
table = xls_file.parse('Sheet1')

In [55]:
table['change'] = table['open']-table['close']
table['long'] = 0
table.head()

,open,high,low,close,open1,high1,low1,close1,open2,high2,...,open9,high9,low9,close9,open10,high10,low10,close10,change,long
2015-11-24 05:00:00,366.25,366.50,366.25,366.25,366.50,366.5,366.25,366.25,366.50,366.5,...,360.50,362.25,360.25,361.75,362.5,362.75,360.00,360.00,0.00,0
2015-11-24 06:00:00,366.25,366.50,366.00,366.25,366.25,366.5,366.25,366.25,366.50,366.5,...,361.50,364.25,361.25,363.25,360.5,362.25,360.25,361.75,0.00,0
2015-11-24 07:00:00,366.50,366.50,366.25,366.50,366.25,366.5,366.00,366.25,366.25,366.5,...,363.50,365.00,362.50,364.50,361.5,364.25,361.25,363.25,0.00,0
2015-11-24 08:00:00,366.25,366.50,366.00,366.25,366.50,366.5,366.25,366.50,366.25,366.5,...,364.50,366.00,364.25,365.75,363.5,365.00,362.50,364.50,0.00,0
2015-11-24 09:00:00,366.25,366.25,366.00,366.00,366.25,366.5,366.00,366.25,366.50,366.5,...,365.75,367.50,365.50,366.75,364.5,366.00,364.25,365.75,0.25,0


In [82]:
def longshort(x):
    if x > 0:
        table['long'] = 1
    elif x < 0:
        table['long'] = -1        

In [88]:
table[table['change'] > 0]

,open,high,low,close,open1,high1,low1,close1,open2,high2,...,open9,high9,low9,close9,open10,high10,low10,close10,change,long
2015-11-24 09:00:00,366.25,366.25,366.00,366.00,366.25,366.50,366.00,366.25,366.50,366.50,...,365.75,367.50,365.50,366.75,364.50,366.00,364.25,365.75,0.25,0
2015-11-24 10:00:00,366.25,366.50,366.00,366.00,366.25,366.25,366.00,366.00,366.25,366.50,...,366.75,367.50,366.25,367.25,365.75,367.50,365.50,366.75,0.25,0
2015-11-24 12:00:00,366.00,366.25,365.25,365.50,366.00,366.25,366.00,366.00,366.25,366.50,...,366.50,366.50,366.25,366.25,366.75,367.25,366.25,366.25,0.50,0
2015-11-24 14:00:00,365.50,365.75,364.00,364.25,365.50,365.75,365.00,365.75,366.00,366.25,...,366.25,366.50,366.25,366.25,366.50,366.50,366.25,366.25,1.25,0
2015-11-24 17:00:00,366.25,366.25,365.00,365.25,365.75,367.00,364.50,366.25,364.50,366.25,...,366.25,366.50,366.00,366.25,366.50,366.50,366.25,366.50,1.00,0
2015-11-24 18:00:00,365.00,365.50,364.50,364.75,366.25,366.25,365.00,365.25,365.75,367.00,...,366.25,366.25,366.00,366.00,366.25,366.50,366.00,366.25,0.25,0
2015-11-24 20:00:00,365.00,365.25,364.00,364.75,364.50,365.25,364.25,364.75,365.00,365.50,...,366.00,366.25,366.00,366.00,366.25,366.50,366.00,366.00,0.25,0
2015-11-25 05:00:00,364.75,364.75,364.25,364.50,364.50,364.50,364.50,364.50,364.25,365.00,...,364.50,366.25,364.00,365.75,365.50,365.75,364.00,364.25,0.25,0
2015-11-25 09:00:00,365.25,365.25,364.75,365.00,365.00,365.25,364.50,365.25,365.00,365.25,...,364.50,365.25,364.25,364.75,365.00,365.50,364.50,364.75,0.25,0
2015-11-25 10:00:00,365.00,365.50,364.75,364.75,365.25,365.25,364.75,365.00,365.00,365.25,...,365.00,365.25,364.00,364.75,364.50,365.25,364.25,364.75,0.25,0
